In [1]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium import webdriver
from IPython.display import clear_output
import string
import json

In [2]:
# Download this here - https://chromedriver.chromium.org/home
driver=webdriver.Chrome(executable_path='C:\Webdrivers\chromedriver.exe')

C:\Users\smara\AppData\Local\Temp\ipykernel_3096\929283111.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=webdriver.Chrome(executable_path='C:\Webdrivers\chromedriver.exe')


In [3]:
services = ['Netflix', 'Hulu', 'HBO MAX', 'Prime Video', 'Disney+']
def get_json(t, y, m):
    ret_json = {}
    
    ## get from json ##
    title = t.translate(str.maketrans('', '', string.punctuation))
    year = y
    media_type = m

    ## construct key for searching ##
    fin = ""
    for i in title.lower().split():
        fin += f"{i}-"
    fin += str(year)
  
    url = f'https://reelgood.com/{media_type}/{fin}'
    driver.get(url)
    
    ## if statement to determine if this item even exists in reelgood ##
    if len(driver.find_elements(By.CLASS_NAME, 'css-1gd9rdl')) == 0:
        
        ## determine if we should remove this item ##
        url = f'https://reelgood.com/{media_type}/{fin}-1'
        driver.get(url)
        ## if this statement is true, that means there were two of these films in the same year with the same name ##
        if len(driver.find_elements(By.CLASS_NAME, 'css-1gd9rdl')) == 0:
            return ret_json
        
        url = f'https://reelgood.com/{media_type}/{fin}'
        driver.get(url)
        
        ## pic ##
        # First get the title of the movie with the correct capitalization
        caps_title = driver.find_element(By.CLASS_NAME, "css-1hbd8x5").text
        img = driver.find_elements(By.CSS_SELECTOR, f'[alt="{caps_title} Poster"]')
        pic = img[0].get_attribute("src") if len(img) > 0 else ""
        ret_json.update({"image": pic})

        ## rank ##
        rank = driver.find_element(By.CLASS_NAME, 'css-r3vd4y').text
        ret_json.update({"rank": (int(rank[1:len(rank)-13]))})

        ## streaming services ##
        sevs = driver.find_elements(By.CLASS_NAME, 'css-16e286y-Span-Name')
        services_list = []
        for i in sevs:
            if i.text in services:
                services_list += [i.text]
        if not services_list:
            ret_json = {}
        else:
            ret_json.update({"services": services_list})
    
    return ret_json
        

In [4]:
# make sure this points at the JSON
with open('media/media.json','r') as f:
    data = json.loads(f.read())


In [ ]:
json_media = {}
media_after_start_year = 0
start_year = 2000
total_media_processed = 0

for key, value in data['media'].items():
    total_media_processed += 1
    if 'startYear' in value and value['startYear'] > start_year:
        media_after_start_year += 1
        
        title = value['originalTitle']
        date = value['startYear']
        media_type = value['titleType'] 
        
        added_json = get_json(title, date, media_type)
        if added_json:
            json_media.update({key: value})
            json_media[key].update(added_json)
    
    clear_output(wait=True)
    print(f"Total media processed: {total_media_processed}")
    print(f"Media after {start_year} processed: {media_after_start_year}")
    print(f"Total json: {len(json_media)}")
    

clear_output(wait=True)
print(f"Total media processed: {total_media_processed}")
print(f"Media after {start_year} processed: {media_after_start_year}")
print(f"Total json: {len(json_media)}")

Total media processed: 72118
Media after 2000 processed: 16
Total json: 2
